In [1]:
import os
import re
import random
import pyodbc
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [2]:
load_dotenv()

True

In [3]:
RANDOM_SEED = 42
DB_NAME = os.getenv("DB_NAME")

In [4]:
def get_db_connection(DB_NAME:str):
    conn = pyodbc.connect(
        f"DRIVER={os.getenv('DB_DRIVER')};"
        f"SERVER={os.getenv('DB_SERVER')};"
        f"DATABASE={DB_NAME};"
        f"UID={os.getenv('DB_UID')};"
        f"PWD={os.getenv('DB_PWD')};"
        f"TrustServerCertificate={os.getenv('TRUST_SERVER_CERTIFICATE')};"
    )
    
    return conn

## I. Xử lý sản phẩm

In [5]:
product_file_name = {
    'dienthoai': 'dienthoai.csv',
    'mayban': 'dienthoaiban.csv',
    'cucgach': 'dienthoaiphothong.csv',
    'dieuhoa': 'dieuhoa.csv',
    'laptop': 'laptop.csv',
    'maydocsach': 'maydocsach.csv',
    'maygiat': 'maygiat.csv',
    'maytinhbang': 'maytinhbang.csv',
    'tivi': 'tivi.csv',
    'tulanh': 'tulanh.csv',
    'camgiamsat': 'cameragiamsat.csv',
    'pc': 'maytinhdeban.csv',
    'mayanh': 'mayanh.csv',
}

product_dataframes = {
    name: pd.read_csv(f'./rawData/{filename}', encoding='utf-8')
    for name, filename in product_file_name.items()
}

### 1. Kiểm tra và loại bỏ các giá trị rỗng, các giá trị trùng lặp

In [6]:
def remove_missing_values(df: pd.DataFrame):
    num_of_row = df.shape[0]
    num_of_missing_rows = df.isnull().any(axis=1).sum()
    print(f"Remove {num_of_missing_rows}/{num_of_row} missing values")
    
    return df.dropna()

def remove_duplicates(df: pd.DataFrame):
    num_of_row = df.shape[0]
    num_of_duplicates = df.duplicated().sum()
    print(f"Remove {num_of_duplicates}/{num_of_row} duplicates")
    
    return df.drop_duplicates()

def remove_missing_and_duplicate_values(name: str, df: pd.DataFrame):
    print(f"Processing dataframe: {name}")
    print("----------------------------------")
    df = remove_missing_values(df)
    df = remove_duplicates(df)
    print("---------------***----------------")
    
    return df

In [7]:
product_dataframes = {
    name: remove_missing_and_duplicate_values(name, df)
    for name, df in product_dataframes.items()
}

Processing dataframe: dienthoai
----------------------------------
Remove 0/230 missing values
Remove 24/230 duplicates
---------------***----------------
Processing dataframe: mayban
----------------------------------
Remove 0/50 missing values
Remove 0/50 duplicates
---------------***----------------
Processing dataframe: cucgach
----------------------------------
Remove 0/18 missing values
Remove 0/18 duplicates
---------------***----------------
Processing dataframe: dieuhoa
----------------------------------
Remove 0/337 missing values
Remove 0/337 duplicates
---------------***----------------
Processing dataframe: laptop
----------------------------------
Remove 0/338 missing values
Remove 0/338 duplicates
---------------***----------------
Processing dataframe: maydocsach
----------------------------------
Remove 0/49 missing values
Remove 7/49 duplicates
---------------***----------------
Processing dataframe: maygiat
----------------------------------
Remove 0/382 missing valu

In [8]:
(
    dienthoai, mayban, cucgach, dieuhoa, 
    laptop, maydocsach, maygiat, maytinhbang, 
    tivi, tulanh, camgiamsat, pc, mayanh
) = product_dataframes.values()

### 2. Đặt lại danh mục cho các sản phẩm

#### 2.1. Điện thoại

In [9]:
dienthoai['Danh mục'].value_counts()

Danh mục
Điện thoại Smartphone         187
Điện Thoại - Máy Tính Bảng     11
Phụ kiện                        6
Root                            2
Name: count, dtype: int64

Kiểm tra sản phẩm thuộc danh mục `Điện Thoại - Máy Tính Bảng`

In [10]:
dienthoai[dienthoai['Danh mục'] == 'Điện Thoại - Máy Tính Bảng']['Tên sản phẩm'].head()

14      Điện thoại Samsung Galaxy A05s - Hàng chính hãng
94     Điện thoại Samsung Galaxy M15 5G - Hàng chính ...
130                Điện thoại Oppo A38 - Hàng chính hãng
163       Điện thoại Xiaomi 14T Pro 5G - Hàng chính hãng
165        Điện thoại Xiaomi Redmi 14C - Hàng chính hãng
Name: Tên sản phẩm, dtype: object

Kiểm tra sản phẩm thuộc danh mục `Root`

In [11]:
dienthoai[dienthoai['Danh mục'] == 'Root']['Tên sản phẩm'].head()

5     Điện Thoại Samsung Galaxy A06 - Đã kích hoạt b...
45    Điện thoại Xiaomi Redmi Note 13 (6GB/128GB) - ...
Name: Tên sản phẩm, dtype: object

Nhận thấy có thể chuyển sản phẩm thuộc 2 danh mục trên vào cùng một danh mục `Điện thoại Smartphone`.

Ngoài ra, đổi tên danh mục `Phụ kiện` thành `Phụ kiện điện thoại` để phân biệt với phụ kiện của các sản phẩm khác.

In [12]:
dienthoai['Danh mục'] = dienthoai['Danh mục'].replace({
    'Điện Thoại - Máy Tính Bảng': 'Điện thoại Smartphone',
    'Root': 'Điện thoại Smartphone',
    'Phụ kiện': 'Phụ kiện điện thoại',
})

In [13]:
dienthoai['Danh mục'].value_counts()

Danh mục
Điện thoại Smartphone    200
Phụ kiện điện thoại        6
Name: count, dtype: int64

#### 2.2. Điện thoại bàn

In [14]:
mayban['Danh mục'].value_counts()

Danh mục
Điện thoại bàn                44
Điện Thoại - Máy Tính Bảng     6
Name: count, dtype: int64

In [15]:
mayban[mayban['Danh mục'] == 'Điện Thoại - Máy Tính Bảng']['Tên sản phẩm'].head()

2     Điện thoại bàn Panasonic KX-TSC11 hàng chính hãng
3     Điện Thoại Bàn Panasonic KX-TSC11 - Hàng Chính...
5     Điện thoại bàn Panasonic KX-TS880 hàng chính hãng
7     Điện thoại bàn Panasonic KX-TGC310 hàng chính ...
10    Điện thoại bàn Panasonic KX-TGD310 hàng chính ...
Name: Tên sản phẩm, dtype: object

Có thể đặt danh mục của tất cả sản phẩm là `Điện thoại bàn`.

In [16]:
mayban['Danh mục'] = 'Điện thoại bàn'

In [17]:
mayban['Danh mục'].value_counts()

Danh mục
Điện thoại bàn    50
Name: count, dtype: int64

#### 2.3. Điện thoại phổ thông

In [18]:
cucgach['Danh mục'].value_counts()

Danh mục
Điện thoại phổ thông          17
Điện Thoại - Máy Tính Bảng     1
Name: count, dtype: int64

Tương tự, có thể đặt tất cả sản phẩm với cùng danh mục là `Điện thoại phổ thông`

In [19]:
cucgach['Danh mục'] = 'Điện thoại phổ thông'

In [20]:
cucgach['Danh mục'].value_counts()

Danh mục
Điện thoại phổ thông    18
Name: count, dtype: int64

#### 2.4. Điều hòa

In [21]:
dieuhoa['Danh mục'].value_counts()

Danh mục
Máy lạnh treo tường        231
Máy lạnh - Máy điều hòa     95
Máy lạnh tủ đứng             9
Máy lạnh âm trần             2
Name: count, dtype: int64

Giữ nguyên tên các danh mục

#### 2.5. Laptop

In [22]:
laptop['Danh mục'].value_counts()

Danh mục
Laptop Truyền Thống                 262
Laptop Gaming                        41
Laptop - Máy Vi Tính - Linh kiện     16
Laptop 2 trong 1                     13
Macbook                               4
Laptop                                1
Root                                  1
Name: count, dtype: int64

Kiểm tra danh mục thuộc `Root`, `Laptop - Máy Vi Tính - Linh kiện`, `Laptop`

In [23]:
print('Sản phẩm thuộc danh mục Root:')
print(laptop[laptop['Danh mục'] == 'Root']['Tên sản phẩm'].head())
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Laptop:')
print(laptop[laptop['Danh mục'] == 'Laptop']['Tên sản phẩm'].head())
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Laptop - Máy Vi Tính - Linh kiện:')
print(laptop[laptop['Danh mục'] == 'Laptop - Máy Vi Tính - Linh kiện']['Tên sản phẩm'].head())

Sản phẩm thuộc danh mục Root:
137    Màn hình K-Vision 27 inch cong IPS tràn viền 7...
Name: Tên sản phẩm, dtype: object
------------------------------------------------------
Sản phẩm thuộc danh mục Laptop:
54    Laptop Asus Expertbook B1502 - B1502CV (Intel ...
Name: Tên sản phẩm, dtype: object
------------------------------------------------------
Sản phẩm thuộc danh mục Laptop - Máy Vi Tính - Linh kiện:
16    DELL LATITUDE E3510 (I5-102104U/ RAM 16GB/ 256...
18    Laptop Dell Latitude 3520 (Core i5-1135G7 | 8G...
23    LAPTOP Thinkpad X1 Carbon Gen 6 | I7 – 8550U |...
27    Laptop HP Pavilion 15-eg2058TU (6K788PA) (i5-1...
67    Máy Tính Xách Tay Màn Hình Cảm Ứng Laptop 2 Tr...
Name: Tên sản phẩm, dtype: object


Nhận thấy, ta có thể phân bổ sản phẩm thuộc 3 danh mục trên vào cùng 1 danh mục `Laptop Truyền Thống`

In [24]:
laptop['Danh mục'] = laptop['Danh mục'].replace({
    'Laptop - Máy Vi Tính - Linh kiện': 'Laptop Truyền Thống',
    'Laptop': 'Laptop Truyền Thống',
    'Root': 'Laptop Truyền Thống',
})

In [25]:
laptop['Danh mục'].value_counts()

Danh mục
Laptop Truyền Thống    280
Laptop Gaming           41
Laptop 2 trong 1        13
Macbook                  4
Name: count, dtype: int64

#### 2.6. Máy đọc sách

In [26]:
maydocsach['Danh mục'].value_counts()

Danh mục
Máy đọc sách                  33
Điện Thoại - Máy Tính Bảng     9
Name: count, dtype: int64

In [27]:
print('Sản phẩm thuộc danh mục Máy đọc sách:')
print(maydocsach[maydocsach['Danh mục'] == 'Máy đọc sách']['Tên sản phẩm'])
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Điện Thoại - Máy Tính Bảng:')
print(maydocsach[maydocsach['Danh mục'] == 'Điện Thoại - Máy Tính Bảng']['Tên sản phẩm'])
print('------------------------------------------------------')

Sản phẩm thuộc danh mục Máy đọc sách:
0     Combo Máy đọc sách All New Kindle Paperwhite 5...
1     Máy đọc sách New Kindle Kids 11th 2022 Kèm bao...
2     Máy đọc sách New Kindle 2024, 16Gb Mới nguyên ...
3     Combo Máy đọc sách All New Kindle Paperwhite 5...
5     Máy đọc sách New Kindle 2024, 16Gb Mới nguyên ...
6     Máy đọc sách Kindle Scribe 2024 Kèm bút Premiu...
7     Máy đọc sách Kindle Colorsoft Signature Editio...
8     Combo Máy đọc sách Amazon Kindle Paperwhite 6 ...
9     Máy đọc sách Amazon Kindle Paperwhite 6 (Gen 1...
10    Máy đọc sách All New Kindle Paperwhite 5 (11th...
12    Máy đọc sách Kindle Paperwhite 5 (8GB) - Hàng ...
14         Máy đọc sách Kindle Scribe - Hàng chính hãng
15             Máy đọc sách Boox Go 6 - Hàng chính hãng
16    Máy đọc sách Boox Go Color 7- Đen - Hàng chính...
17          Máy đọc sách Boox Go 10.3 - Hàng chính hãng
18    Máy đọc sách Pocketbook Era Color- Hàng chính ...
21    Combo Máy đọc sách Kindle Scribe kèm bút – thế...
22        

Có một số sản phẩm `máy tính bảng` bị lẫn trong danh mục `Máy đọc sách`. 

Ta thực hiện gộp sản phẩm thuộc 2 danh mục này vào danh mục `Máy đọc sách`. Sau đó lọc ra các sản phẩm `máy tính bảng` rồi cho vào danh mục `Máy tính bảng`

In [28]:
maydocsach['Danh mục'] = 'Máy đọc sách'

maydocsach.loc[maydocsach['Tên sản phẩm'].str.contains('máy tính bảng', case=False), 'Danh mục'] = 'Máy tính bảng'

In [29]:
maydocsach['Danh mục'].value_counts()

Danh mục
Máy đọc sách     39
Máy tính bảng     3
Name: count, dtype: int64

#### 2.7. Máy giặt

In [30]:
maygiat['Danh mục'].value_counts()

Danh mục
Máy giặt cửa ngang    160
Máy giặt cửa trên     118
Máy giặt               90
Tháp giặt sấy          12
Máy giặt mini           2
Name: count, dtype: int64

Giữ nguyên các danh mục trên

#### 2.8. Máy tính bảng

In [31]:
maytinhbang['Danh mục'].value_counts() 

Danh mục
Máy tính bảng                 24
Điện Thoại - Máy Tính Bảng     5
Root                           4
Name: count, dtype: int64

Gom tất cả sản phẩm cùng chung 1 danh mục `Máy tính bảng`

In [32]:
maytinhbang['Danh mục'] = 'Máy tính bảng'

In [33]:
maytinhbang['Danh mục'].value_counts()

Danh mục
Máy tính bảng    33
Name: count, dtype: int64

#### 2.9. Tivi

In [34]:
tivi['Danh mục'].value_counts()

Danh mục
Smart Tivi - Android Tivi    117
Tivi 4K                       91
Điện Tử - Điện Lạnh           19
Tivi thường (LED)             13
Tivi OLED                     11
Root                           8
Tivi QLED                      5
Tivi cao cấp                   5
Internet Tivi                  4
Name: count, dtype: int64

Kiểm tra sản phẩm thuộc danh mục `Điện Tử - Điện Lạnh` và `Root`

In [35]:
print('Sản phẩm thuộc danh mục Điện Tử - Điện Lạnh:')
print(tivi[tivi['Danh mục'] == 'Điện Tử - Điện Lạnh']['Tên sản phẩm'])
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Root:')
print(tivi[tivi['Danh mục'] == 'Root']['Tên sản phẩm'])
print('------------------------------------------------------')

Sản phẩm thuộc danh mục Điện Tử - Điện Lạnh:
26     Màn hình LED Arirang 43 inch UHD 4k AR-43UM01 ...
37                 Smart Tivi Casper HD 32 inch 32HX6200
38             Smart Tivi Asanzo Full HD 43 inch 43AS560
39                   Tivi LED Asanzo HD 25 inch 25S200T2
74     Google Tivi Sony 4K 43 inch KD-43X75K - Hàng c...
78            Android Tivi OLED Sony 4K 55 inch KD-55A8F
79      Tivi LED Asanzo 25 inch 25T350 - Hàng Chính Hãng
104    Tivi xách tay LG StanbyME Go 27LX5QKNA 27 inch...
107    Smart Tivi LG 55 Inch 4K 55UQ7050PSA - Hàng ch...
110      Tivi 32IN HÀNG CHÍNH HÃNG NHẬP KHẨU TỪ HÀN QUỐC
119                   Tivi LED Asanzo HD 32 inch 32AT120
148          Android TV K-Elec 65UK885V - Hàng nhập khẩu
157                   Tivi LED Asanzo HD 32 inch 32AH102
158         Smart Tivi QLED Samsung 4K 65 inch QA65Q7FNA
159    Smart Tivi Cong QLED Samsung 4K 55 inch QA55Q8CNA
185    Smart ti vi Asanzo 55 AG800K- màn hình cường l...
232    Google Tivi Sony 4K 75 inch KD.75X77

Tiến hành chia danh mục:
- Các sản phẩm thuộc danh mục `Điện Tử - Điện Lạnh` và `Root` có thể được phân bổ vào các danh mục: `Tivi OLED`, `Tivi QLED`, `Smart Tivi - Android Tivi`, `Tivi thường (LED)`, `Tivi 4K` tùy thuộc vào tên của sản phẩm đó.

    - Nếu tên sản phẩm chứa từ `oled` $\rightarrow$ Sản phẩm thuộc danh mục `Tivi OLED`
    - Nếu tên sản phẩm chứa từ `qled` $\rightarrow$ Sản phẩm thuộc danh mục `Tivi QLED`
    - Nếu tên sản phẩm chứa từ `smart` hoặc `android` $\rightarrow$ Sản phẩm thuộc danh mục `Smart Tivi - Android Tivi`
    - Nếu tên sản phẩm chứa từ `led` $\rightarrow$ Sản phẩm thuộc danh mục `Tivi thường (LED)`
    - Nếu tên sản phẩm chứa từ `4k` $\rightarrow$ Sản phẩm thuộc danh mục `Tivi 4K`


In [36]:
def assign_category_to_tivi(df: pd.DataFrame, keywords: str, category: str):
    mask = df['Danh mục'].isin(['Điện Tử - Điện Lạnh', 'Root']) & df['Tên sản phẩm'].str.contains(keywords, case=False)
    df.loc[mask, 'Danh mục'] = category

In [37]:
assign_category_to_tivi(tivi, 'oled', 'Tivi OLED')
assign_category_to_tivi(tivi, 'qled', 'Tivi QLED')
assign_category_to_tivi(tivi, 'smart|android', 'Smart Tivi - Android Tivi')
assign_category_to_tivi(tivi, 'led', 'Tivi thường (LED)')
assign_category_to_tivi(tivi, '4k', 'Tivi 4K')

In [38]:
tivi['Danh mục'].value_counts()

Danh mục
Smart Tivi - Android Tivi    129
Tivi 4K                       94
Tivi thường (LED)             19
Tivi OLED                     13
Tivi QLED                      7
Tivi cao cấp                   5
Internet Tivi                  4
Điện Tử - Điện Lạnh            2
Name: count, dtype: int64

Kiểm tra các sản phẩm chưa được tái phân loại trong danh mục `Điện Tử - Điện Lạnh`

In [39]:
tivi[tivi['Danh mục'] == 'Điện Tử - Điện Lạnh']['Tên sản phẩm'].head()

104    Tivi xách tay LG StanbyME Go 27LX5QKNA 27 inch...
110      Tivi 32IN HÀNG CHÍNH HÃNG NHẬP KHẨU TỪ HÀN QUỐC
Name: Tên sản phẩm, dtype: object

Phân loại các sản phẩm trên vào danh mục `Smart Tivi - Android Tivi`

In [40]:
tivi.loc[tivi['Danh mục'] == 'Điện Tử - Điện Lạnh', 'Danh mục'] = 'Smart Tivi - Android Tivi'

In [41]:
tivi['Danh mục'].value_counts()

Danh mục
Smart Tivi - Android Tivi    131
Tivi 4K                       94
Tivi thường (LED)             19
Tivi OLED                     13
Tivi QLED                      7
Tivi cao cấp                   5
Internet Tivi                  4
Name: count, dtype: int64

#### 2.10. Tủ lạnh

In [42]:
tulanh['Danh mục'].value_counts()

Danh mục
Tủ lạnh nhiều cửa       164
Tủ lạnh                  78
Điện Tử - Điện Lạnh      68
Tủ lạnh side by side     53
Tủ lạnh mini             37
Name: count, dtype: int64

In [43]:
tulanh[tulanh['Danh mục'] == 'Điện Tử - Điện Lạnh']['Tên sản phẩm'].head()

19    Tủ lạnh Toshiba Inverter 233 lít GR-RT303WE-PM...
20    Tủ lạnh Toshiba GR-RT234WE-PMV(52) Inveter 202...
32    Tủ lạnh Panasonic NR-BA190PPVN - Hàng Chính Hã...
36    Tủ lạnh Inverter AQR-T239FA(HB) - Hàng chính h...
53    TỦ LẠNH TOSHIBA GR-RT252WE-PMV(52) 194 Lít - h...
Name: Tên sản phẩm, dtype: object

Chuyển sản phẩm thuộc danh mục `Điện Tử - Điện Lạnh` vào danh mục `Tủ lạnh`

In [44]:
tulanh['Danh mục'] = tulanh['Danh mục'].replace({
    'Điện Tử - Điện Lạnh': 'Tủ lạnh',
})

In [45]:
tulanh['Danh mục'].value_counts()

Danh mục
Tủ lạnh nhiều cửa       164
Tủ lạnh                 146
Tủ lạnh side by side     53
Tủ lạnh mini             37
Name: count, dtype: int64

#### 2.11. Camera giám sát

In [46]:
camgiamsat['Danh mục'].value_counts()

Danh mục
Camera IP                   283
Máy Ảnh - Máy Quay Phim      78
Phụ Kiện Camera Giám Sát     17
Đầu Ghi Hình Camera          11
Root                          8
Hệ Thống Camera Giám Sát      7
Camera Quan Sát Analog        2
Name: count, dtype: int64

Kiểm tra sản phẩm thuộc danh mục `Camera IP`, `Máy Ảnh - Máy Quay Phim`, `Root`

In [47]:
print('Sản phẩm thuộc danh mục Camera IP:')
print(camgiamsat[camgiamsat['Danh mục'] == 'Camera IP']['Tên sản phẩm'])
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Máy Ảnh - Máy Quay Phim:')
print(camgiamsat[camgiamsat['Danh mục'] == 'Máy Ảnh - Máy Quay Phim']['Tên sản phẩm'])
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Root:')
print(camgiamsat[camgiamsat['Danh mục'] == 'Root']['Tên sản phẩm'])
print('------------------------------------------------------')

Sản phẩm thuộc danh mục Camera IP:
0      Camera Wifi Ezviz 2 Mắt Ngoài Trời H9C 3K 6MP/...
2      Camera Wifi EZVIZ H3C 2MP Có Màu Ban Đêm, H3C ...
3      Camera Yoosee Wifi Ngoài Trời 2 Mắt Xem Cùng L...
5      Camera Yoosee Wifi Ngoài Trời 2 Mắt Xem Cùng L...
6      Camera Yoosee Ngoài Trời F50 3 MẮT MÁY BAY ĐỘC...
                             ...                        
418               Camera Ezviz C6N 1080p Hàng Chính Hãng
420    Camera ip wifi xoay 360 độ Orikon - Hàng chính...
421    DS-2CD1327G0-L CAMERA IP HIKVISION COLORVU LIT...
423    Camera IP 1080P Xiaomi Mi Home Magnetic Mount ...
424    Camera IP Hồng Ngoại 2MP Hanwha Techwin Wisene...
Name: Tên sản phẩm, Length: 283, dtype: object
------------------------------------------------------
Sản phẩm thuộc danh mục Máy Ảnh - Máy Quay Phim:
1      Camera Wifi EZVIZ H3C 4MP 3K - H3C 3M, Có Màu ...
4      Camera Wi-Fi TP-Link Tapo C200 1080P (2MP) An ...
18     Camera IP Wifi Trong Nhà EZVIZ C6N 1080p - Hàn...
22     Camera Wi

Nhìn chung, các sản phẩm có thể phân bổ theo các danh mục như sau:

- Camera IP - Camera Wifi: Gồm các sản phẩm thuộc danh mục `Camera IP`, `Máy Ảnh - Máy Quay Phim` và `Root`
- Phụ Kiện Camera Giám Sát: Gồm các sản phẩm thuộc danh mục `Phụ Kiện` và `Root`
    - Với danh mục `Root`:
        - Nếu tên sản phẩm chứa từ `ip` hoặc `wifi` $\rightarrow$
 Sản phẩm thuộc danh mục `Camera IP - Camera Wifi`
        - Các sản phẩm còn lại thuộc danh mục `Phụ Kiện Camera Giám Sát`
- Đầu Ghi Hình Camera
- Camera Quan Sát Analog
- Hệ Thống Camera Giám Sát

In [48]:
camgiamsat['Danh mục'] = camgiamsat['Danh mục'].replace({
    'Camera IP': 'Camera IP - Camera Wifi',
    'Máy Ảnh - Máy Quay Phim': 'Camera IP - Camera Wifi',
})

camgiamsat.loc[
    (camgiamsat['Danh mục'] == 'Root') & (camgiamsat['Tên sản phẩm'].str.contains('ip|wifi', case=False)), 
    'Danh mục'
    ] = 'Camera IP - Camera Wifi'

camgiamsat['Danh mục'] = camgiamsat['Danh mục'].replace({
    'Root': 'Phụ Kiện Camera Giám Sát',
})

In [49]:
camgiamsat['Danh mục'].value_counts()

Danh mục
Camera IP - Camera Wifi     365
Phụ Kiện Camera Giám Sát     21
Đầu Ghi Hình Camera          11
Hệ Thống Camera Giám Sát      7
Camera Quan Sát Analog        2
Name: count, dtype: int64

#### 2.12. Máy tính để bàn

In [50]:
pc['Danh mục'].value_counts()

Danh mục
Máy Tính Bộ Thương Hiệu             89
Mini PC                             22
Laptop - Máy Vi Tính - Linh kiện    10
Máy Tính All in one                 10
Towers - Máy Chủ - Server           10
Máy Tính Để Bàn Lắp Ráp              6
PC Gaming                            3
Root                                 2
PC - Máy Tính Bộ                     1
Name: count, dtype: int64

Kiểm tra sản phẩm thuộc các danh mục `Laptop - Máy Vi Tính - Linh Kiện`, `Root`, `PC - Máy Tính Bộ`

In [51]:
print('Sản phẩm thuộc danh mục Laptop - Máy Vi Tính - Linh Kiện:')
print(pc[pc['Danh mục'] == 'Laptop - Máy Vi Tính - Linh kiện']['Tên sản phẩm'])
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục Root:')
print(pc[pc['Danh mục'] == 'Root']['Tên sản phẩm'])
print('------------------------------------------------------')

print('Sản phẩm thuộc danh mục PC - Máy Tính Bộ:')
print(pc[pc['Danh mục'] == 'PC - Máy Tính Bộ']['Tên sản phẩm'])
print('------------------------------------------------------')

Sản phẩm thuộc danh mục Laptop - Máy Vi Tính - Linh Kiện:
0      Máy Tính Siêu Nhỏ Ultra Mini Morefine ZX01 Plu...
17     Máy Tính Để Bàn Dell Optiplex 3040 Core i7 670...
26     Máy tính để bàn nhỏ gọn ThinkCentre  LENOVO ( ...
28     Máy tính văn phòng C800 (Core i5/ SSD 240 GB/ ...
43     Máy Tính Siêu Nhỏ Ultra Mini Morefine ZX01 Plu...
80     Máy tính văn phòng mini Intel NUC7CJYH - Chưa ...
81     Máy tính văn phòng mini Intel NUC6CAYS - Chưa ...
82     Máy Tính Đồng Bộ DELL OPTIPLEX 7010 (Intel i5,...
101    Máy Tính Để Bàn Dell Optiplex 3050 micro nhỏ x...
123    Máy tính văn phòng C500 (Core i3/ SSD 120GB/ R...
Name: Tên sản phẩm, dtype: object
------------------------------------------------------
Sản phẩm thuộc danh mục Root:
109    Máy tính Dell workstation Precision T5810 cpu ...
140    Máy tính Dell workstation Precision T5810 cpu ...
Name: Tên sản phẩm, dtype: object
------------------------------------------------------
Sản phẩm thuộc danh mục PC - Máy Tính Bộ:
83    Má

Phân bổ sản phẩm và các danh mục như sau:
- Máy tính đồng bộ: Gồm sản phẩm thuộc danh mục `Máy Tính Bộ Thương Hiệu`, `Laptop - Máy Vi Tính - Linh kiện`, `Root`, `PC - Máy Tính Bộ`
- Mini PC: Gồm sản phẩm thuộc danh mục `Mini PC`, `Laptop - Máy Vi Tính - Linh kiện`
- Towers - Máy Chủ - Server
- Máy Tính All in one
- Máy Tính Để Bàn Lắp Ráp
- PC Gaming

In [52]:
pc['Danh mục'] = pc['Danh mục'].replace({
    'Máy Tính Bộ Thương Hiệu': 'Máy tính đồng bộ',
    'Root': 'Máy tính đồng bộ',
    'PC - Máy Tính Bộ': 'Máy tính đồng bộ',
})

pc.loc[
    (pc['Danh mục'] == 'Laptop - Máy Vi Tính - Linh kiện') & (pc['Tên sản phẩm'].str.contains('mini|siêu nhỏ')),
    'Danh mục'
] = 'Mini PC'

pc['Danh mục'] = pc['Danh mục'].replace({
    'Laptop - Máy Vi Tính - Linh kiện': 'Máy tính đồng bộ',
})

In [53]:
pc['Danh mục'].value_counts()

Danh mục
Máy tính đồng bộ             100
Mini PC                       24
Máy Tính All in one           10
Towers - Máy Chủ - Server     10
Máy Tính Để Bàn Lắp Ráp        6
PC Gaming                      3
Name: count, dtype: int64

#### 2.13. Máy ảnh

In [54]:
mayanh['Danh mục'].value_counts()

Danh mục
Máy Ảnh Mirrorless                   4
Máy Ảnh Compact - Máy Ảnh Du Lịch    4
Máy Ảnh - Máy Quay Phim              3
Máy Ảnh Lấy Liền                     3
Máy Chụp Ảnh Phim                    1
Name: count, dtype: int64

Giữ nguyên danh mục như cũ

### 3. Tạo data cho hệ thống dựa theo Cơ sở dữ liệu đã chuẩn bị

Gộp các sản phẩm vào dataframe `products`

In [55]:
products = pd.concat(
    [
        dienthoai, mayban, cucgach, dieuhoa, 
        laptop, maydocsach, maygiat, maytinhbang, 
        tivi, tulanh, camgiamsat, pc, mayanh
    ],
    ignore_index=True
)

In [56]:
products.columns.unique()

Index(['Id', 'Tên sản phẩm', 'Thương hiệu', 'Danh mục', 'Thông số kỹ thuật',
       'Phiên bản', 'Mô tả', 'Hình ảnh'],
      dtype='object')

Kiểm tra trùng lặp Id sản phẩm (Do các sản phẩm thuộc danh mục khác nhau có thể trùng Id).

In [57]:
print('Số bản ghi trong products có Id trùng nhau:', end=' ')
print(products['Id'].duplicated().sum())

Số bản ghi trong products có Id trùng nhau: 17


Loại bỏ các bản ghi trùng Id

In [58]:
products.drop_duplicates(subset=['Id'], inplace=True)

#### 3.1. Tạo dataframe `category` (id, name)

In [59]:
category_rows = []
for index, row in enumerate(products['Danh mục'].unique()):
    category_rows.append({
        'id': index + 1,
        'name': row,
    })

category_df = pd.DataFrame(category_rows)


In [60]:
category_df.head()

,id,name
0,1,Điện thoại Smartphone
1,2,Phụ kiện điện thoại
2,3,Điện thoại bàn
3,4,Điện thoại phổ thông
4,5,Máy lạnh treo tường


### 3.2. Tạo dataframe `product` và các dataframe liên quan đến biến thể sản phẩm

Xây dựng:

|`product`|`attribute`|`attribute_value`|`product_variant`|`attribute_variant`|
|-------|---------|---------------|---------------|--------|
|id|id|id|id|attribute_value_id|
|category_id|name|attribute_id|product_id|attribute_id|
|name||value|price|product_variant_id|
|description|||original_price||
|specification|||profit||
|image_url|||sku||
|brand|||stock_quantity||
||||sold_quantity||

Tạo từ điển `category_dict` để tạo ánh xạ từ `tên danh mục` sang `id danh mục` (phục vụ cho việc tạo bảng `product`)

In [61]:
category_dict = dict(zip(category_df['name'], category_df['id']))

#### 3.2. Tạo dataframe `attribute` (id, name)

Thực hiện phân tách các `thuộc tính (attribute)` của sản phẩm trong cột `phiên bản` (gồm các thuộc tính dùng để phân biệt các phiên bản với nhau)

Sau khảo sát, việc phân tách thuộc tính như sau:
Với:
- Điện thoại: Gồm 2 thuộc tính: Màu và Dung lượng
- Camera giám sát: Gồm:
    - Model: Trích từ các thuộc tính `model`, `model camera`, `lựa chọn mẫu` trong cột `phiên bản`
    - Độ phân giải: Trích từ `độ phân giải`
    - Màu
    - Lựa chọn
- Điện thoại bàn:  Gồm thuộc tính Màu
- Điện thoại phổ thông: Gồm thuộc tính Màu
- Điều hòa: Gồm:
    - Công suất: Trích từ các thuộc tính `Phân loại công suất`, `công suất làm lạnh`, `công suất lạnh`
    - Bảo hành: Trích từ các thuộc tính `Bảo hành 2 năm`, `bảo hành toàn quốc`
- Laptop: Gồm các thuộc tính: Dung lượng, Màu, Chip, Hệ điều hành, Màn (trích từ `Loại màn`)
- Máy ảnh: Gồm các thuộc tính: Màu và Lựa chọn
- Máy đọc sách: Gồm:
    - Màu: Trích từ các thuộc tính về `màu` và `Màu bao da`
    - Dung lượng
    - Bút đi kèm: Trích từ `loại bút đi kèm`
    - Lựa chọn
- Máy giặt: Gồm thuộc tính Màu
- Máy tính bảng: Gồm các thuộc tính: Màu, Dung lượng, Lựa chọn
- Máy tính để bàn: Gồm các thuộc tính: Dung lượng, Cấu hình, Lựa chọn
- Tivi: Gồm các thuộc tính: Kích cỡ (Trích từ `Kích cỡ màn hình`) và Lựa chọn
- Tủ lạnh: Gồm 2 thuộc tính: Màu, Lựa chọn

In [62]:
def toPascalCase(x: str):
    return ' '.join(word.capitalize() for word in x.lower().split())

def rename_attribute(attribute: str, value: str):
    attribute = attribute.lower()
    value = value.lower()

    if any(attr in attribute for attr in ['màu', 'colour', 'color']) \
        and all(not char.isdigit() for char in value):
        return 'Màu'
    
    if any(attr in attribute for attr in ['dung lượng', 'ram', 'memory', 'storage']):
        return 'Dung lượng'
    
    if any(attr in attribute for attr in ['model', 'model camera', 'lựa chọn mẫu', 'mẫu']):
        return 'Model'
    
    if any(attr in attribute for attr in ['độ phân giải', 'phân giải', 'resolution']):
        return 'Độ phân giải'
    
    if any(attr in attribute for attr in ['công suất', 'power']):
        return 'Công suất'

    if any(attr in attribute for attr in ['bảo hành', 'warranty']):
        return 'Bảo hành'
    
    if any(attr in attribute for attr in ['chip', 'cpu', 'vi xử lý', 'processor']):
        return 'Chip'
    
    if any(attr in attribute for attr in ['hệ điều hành', 'os', 'operating system', 'win']):
        return 'Hệ điều hành'
    
    if any(attr in attribute for attr in ['màn', 'display', 'screen']):
        return 'Màn hình'
    
    if any(attr in attribute for attr in ['bút', 'pen']):
        return 'Bút đi kèm'

    return 'Lựa chọn'

Trích xuất các phiên bản từ từng sản phẩm

In [63]:
def extract_options(versions: str):

    if '=' not in versions:
        return [
            {
                'attrs': ['Loại'],
                'values': ['Mặc Định'],
                'price': int(versions.split()[0].strip())
            }
        ]

    options = []

    lines = versions.strip().split('\n')

    for line in lines:
        attributes, price_str = line.rsplit('=', 1)
        
        price = int(price_str.strip().split()[0])

        split_attributes = attributes.strip().split('$$')

        attrs, values = [], []

        for pair in split_attributes:
            attribute, value = map(str.strip, pair.split(':', 1))
            
            value = toPascalCase(value)
            attribute = rename_attribute(attribute.lower(), value)

            attrs.append(attribute)
            values.append(value)

        options.append({
            'attrs': attrs,
            'values': values,
            'price': price
        })

    return options

Trích xuất các thuộc tính (thuộc tính phân biệt các phiên bản)

In [64]:
def extract_attributes(df: pd.DataFrame):
    attrs = {}

    rows = df['Phiên bản'].astype(str).str.strip()

    for versions in rows:

        options = extract_options(versions)

        for option in options:
            for attr, value in zip(option['attrs'], option['values']):
                
                attrs.setdefault(attr, set()).add(value)

    return attrs

In [65]:
attributes = extract_attributes(products)
attributes = sorted(attributes.items(), key=lambda x: (x[0], x[1]))

attribute_rows = []

for index, (name, _) in enumerate(attributes):
    attribute_rows.append({
        'id': index + 1,
        'name': name,
    })

attribute_df = pd.DataFrame(attribute_rows)

In [66]:
attribute_df.head()

,id,name
0,1,Bút đi kèm
1,2,Bảo hành
2,3,Chip
3,4,Công suất
4,5,Dung lượng


Tạo từ điển `attribute_dict` để ánh xạ từ `tên thuộc tính` sang `id thuộc tính` (phục vụ cho việc tạo bảng `attribute_value`)

In [67]:
attribute_dict = dict(zip(attribute_df['name'], attribute_df['id']))

#### 3.3. Tạo dataframe `attribute_value` (id, attribute_id, value)

In [68]:
attribute_rows = []

attribute_value_id_counter = 1

for index, (name, values) in enumerate(attributes):
    for value in values:
        
        attribute_rows.append({
            'id': attribute_value_id_counter,
            'attribute_id': index + 1,
            'value': value,
        })

        attribute_value_id_counter += 1

attribute_value_df = pd.DataFrame(attribute_rows)

In [69]:
attribute_value_df

,id,attribute_id,value
0,1,1,Combo Premium Pen + Cover Fabric Chính Hãng
1,2,1,Premium Pen
2,3,1,Basic Pen
3,4,2,3 Tháng
4,5,2,Mpac12b Cánh Đảo
...,...,...,...
710,711,12,S2xp 6mp
711,712,12,2mp/1080p
712,713,12,S3dp 5.0mp
713,714,12,2k - 3mp


Tạo từ điển `attribute_value_dict` để ánh xạ từ `giá trị thuộc tính` sang `id giá trị thuộc tính`

In [70]:
attribute_value_dict = dict(zip(attribute_value_df['value'], attribute_value_df['id']))

#### 3.4. Tạo dataframe `product`, `product_variant` và `attribute_variant`

Chuẩn hóa câu có dấu về câu tiếng anh (để tạo SKU)

In [71]:
def normalize_vietnamese_string(s: str):
    vietnamese_chars = {
        'à': 'a', 'á': 'a', 'ả': 'a', 'ã': 'a', 'ạ': 'a',
        'ă': 'a', 'ằ': 'a', 'ắ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ặ': 'a',
        'â': 'a', 'ầ': 'a', 'ấ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ậ': 'a',
        'đ': 'd',
        'è': 'e', 'é': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ẹ': 'e',
        'ê': 'e', 'ề': 'e', 'ế': 'e', 'ể': 'e', 'ễ': 'e', 'ệ': 'e',
        'ì': 'i', 'í': 'i', 'ỉ': 'i', 'ĩ': 'i', 'ị': 'i',
        'ò': 'o', 'ó': 'o', 'ỏ': 'o', 'õ': 'o', 'ọ': 'o',
        'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ổ': 'o', 'ỗ': 'o', 'ộ': 'o',
        'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ở': 'o', 'ỡ': 'o', 'ợ': 'o',
        'ù': 'u', 'ú': 'u', 'ủ': 'u', 'ũ': 'u', 'ụ': 'u',
        'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ử': 'u', 'ữ': 'u', 'ự': 'u',
        'ỳ': 'y', 'ý': 'y', 'ỷ': 'y', 'ỹ': 'y', 'ỵ': 'y'
    }

    s = s.lower()

    for vn_char, latin_char in vietnamese_chars.items():
        s = s.replace(vn_char, latin_char)
    
    s = re.sub(r'[^a-z ]','', s)

    return s

Tạo mã SKU = 2 ký tự đầu của `thương hiệu` + ký tự đầu tiên của các từ của `danh mục` + mã biến thể

In [72]:
def create_sku(brand, category, variant_id):
    brand = normalize_vietnamese_string(brand)
    category = normalize_vietnamese_string(category)

    sku = f"{brand[:2].upper()}-{''.join(map(lambda x: x[0].upper(), category.split()))}-{variant_id}"

    return sku

Tạo giá nhập cho các biến thể

In [73]:
def create_original_price(price: float):
    mean = 0.8
    std_dev = 0.05

    original_price = price * random.gauss(mean, std_dev)
    original_price = max(original_price, price * 0.5)

    return round(original_price)

Tạo từ điển `option_to_variant_id` để ánh xạ từ `phiên bản` sang `id của biến thể` (phục vụ tạo bảng `feedback` sau này)

Tạo từ điển `old_product_id_to_new_product_id` để ánh xạ từ `id` của bảng sản phẩm cũ sang `id` của bảng sản phẩm mới

In [74]:
option_to_variant_id = {}

old_product_id_to_new_product_id = {}

Tạo đồng thời ba bảng `product`, `product_variant`, `attribute_variant`

In [75]:
product_rows = []
product_variant_rows = []
attribute_variant_rows = []

variant_id_counter = 1

for index, row in tqdm(products.iterrows(), total=products.shape[0], desc="Processing products", unit="rows", colour="green"):
    versions = str(row['Phiên bản']).strip()

    options = extract_options(versions)

    product_id = index

    category_id = category_dict[row['Danh mục']]

    product_rows.append({
        'id': product_id,
        'category_id': category_id,
        'name': row['Tên sản phẩm'],
        'description': row['Mô tả'],
        'specification': row['Thông số kỹ thuật'],
        'image_url': row['Hình ảnh'],
        'brand': row['Thương hiệu'],
    })
    
    random.seed(RANDOM_SEED)
    
    for option in options:
        
        stock_quantity = random.randint(0, 120)
        
        price = option['price']

        original_price = create_original_price(price)

        profit = price - original_price

        sku = create_sku(row['Thương hiệu'], row['Danh mục'], variant_id_counter)

        sold_quantity = random.randint(0, 100)

        product_variant_rows.append({
            'id': variant_id_counter,
            'product_id': product_id,
            'price': price,
            'original_price': original_price,
            'profit': profit,
            'sku': sku,
            'stock_quantity': stock_quantity,
            'sold_quantity': sold_quantity,
        })

        for attr, val in zip(option['attrs'], option['values']):
            attribute_value_id = attribute_value_dict[val]
            attrirbute_id = attribute_dict[attr]

            attribute_variant_rows.append({
                'product_variant_id': variant_id_counter,
                'attribute_id': attrirbute_id,
                'attribute_value_id': attribute_value_id,
            })

        key = (
            int(row['Id']),
            tuple(sorted(zip(option['attrs'], option['values']))),
        )

        option_to_variant_id[key] = variant_id_counter

        old_product_id_to_new_product_id[int(row['Id'])] = product_id

        variant_id_counter += 1

product_df = pd.DataFrame(product_rows)
product_variant_df = pd.DataFrame(product_variant_rows)
attribute_variant_df = pd.DataFrame(attribute_variant_rows)

Processing products: 100%|██████████| 2636/2636 [00:00<00:00, 8865.52rows/s]


In [76]:
product_variant_df.head()

,id,product_id,price,original_price,profit,sku,stock_quantity,sold_quantity
0,1,0,5600000,4832436,767564,SA-DTS-1,81,31
1,2,0,5650000,4819208,830792,SA-DTS-2,28,17
2,3,1,22990000,19838876,3151124,SA-DTS-3,81,31
3,4,1,22990000,19609485,3380515,SA-DTS-4,28,17
4,5,1,21990000,19036604,2953396,SA-DTS-5,94,69


# II. Xử lý phản hồi của người dùng về sản phẩm và phản hồi của nhân viên hỗ trợ khách hàng

### 1. Tạo dataframe `feeback` (id, customer_id, product_id, product_variant_id, rating, comment, created_at)

#### 1.1. Tiền xử lý

In [77]:
feedback_file_name = {
    'camera_fb': 'camera_fb.csv',
    'dienthoai_fb': 'dienthoai_fb.csv',
    'mayban_fb': 'dienthoaiban_fb.csv',
    'cucgach_fb': 'dienthoaiphothong_fb.csv',
    'dieuhoa_fb': 'dieuhoa_fb.csv',
    'laptop_fb': 'laptop_fb.csv',
    'mayanh_fb': 'mayanh_fb.csv',
    'maydocsach_fb': 'maydocsach_fb.csv',
    'maygiat_fb': 'maygiat_fb.csv',
    'maytinhbang_fb': 'maytinhbang_fb.csv',
    'tivi_fb': 'tivi_fb.csv',
    'tulanh_fb': 'tulanh_fb.csv',
    'pc_fb': 'maytinhdeban_fb.csv',
}

feedback_dataframes = {
    name: pd.read_csv(f'./feedback/{filename}', encoding='utf-8')
    for name, filename in feedback_file_name.items()
}

feedback_dataframes = {
    name: remove_duplicates(df)
    for name, df in feedback_dataframes.items()
}

Remove 45/414 duplicates
Remove 182/597 duplicates
Remove 0/23 duplicates
Remove 0/14 duplicates
Remove 0/204 duplicates
Remove 0/52 duplicates
Remove 0/2 duplicates
Remove 15/112 duplicates
Remove 0/107 duplicates
Remove 30/90 duplicates
Remove 0/283 duplicates
Remove 0/308 duplicates
Remove 0/0 duplicates


Gộp các feedback thành 1 dataframe duy nhất

In [78]:
feedbacks = pd.concat([
    df for df in feedback_dataframes.values()
], ignore_index=True)

In [79]:
feedbacks.head()

,feedback_id,product_id,customer_id,rating,content,time,variant,image_url
0,5278285,47868431,17145651,5,Đã mua gói hàng rất cẩn thận cảm thấy hài lòng...,2020-10-19 13:34:13,NaN,https://salt.tikicdn.com/ts/review/83/90/16/ac...
1,5397823,47868431,12180385,5,"Sản phẩm dễ cài đặt, nhiều tính năng, độ phân ...",2020-10-29 15:35:34,NaN,https://salt.tikicdn.com/ts/review/9d/f9/73/53...
2,5437697,47868431,11096412,5,"hình rõ, app trên Android lẫn iOS chưa ổn định...",2020-11-02 11:23:52,NaN,https://salt.tikicdn.com/ts/review/18/88/78/84...
3,20089032,47868431,9883981,5,NaN,2025-02-28 21:09:02,NaN,NaN
4,5950422,47868431,709976,5,"giá rẻ, chất lượng tốtgiá rẻ, chất lượng tốtgi...",2020-12-04 23:53:11,NaN,NaN


#### 1.2. Tạo dataframe

Khởi tạo hàm `extract_version_to_variant_id` để lấy `id biến thể` khi biết `phiên bản`

In [80]:
def mapping_option_to_variant_id(product_id: int, option: str):
    if pd.isna(option) or option == '' or option == 'nan':
        return None
    
    attributes = option.split('$$')
    attrs, values = [], []

    for pair in attributes:
        attr_value = pair.split(':')
        attr, value = attr_value[0].strip().lower(), attr_value[-1].strip().lower()

        attr = rename_attribute(attr, value)
        value = toPascalCase(value)

        attrs.append(attr)
        values.append(value)

    key = (
        product_id,
        tuple(sorted(zip(attrs, values))),
    )

    return option_to_variant_id.get(key, None)

In [81]:
def mapping_old_product_id_to_new_product_id(old_product_id: str):
    return old_product_id_to_new_product_id.get(old_product_id, None)

In [82]:
def random_date(start_date: str, end_date: str = datetime.now().strftime('%Y-%m-%d')):
    try:
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    except ValueError:
        start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

    try:        
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError:
        end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')


    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_date = start_date + timedelta(days=random_days)

    return random_date.strftime('%Y-%m-%d %H:%M:%S')

Lấy id khách hàng (`customer_id`) và ngày tạo tài khoản (`created_at`) của các khách hàng với điều kiện tài khoản đó không bị cấm bởi hệ thống

In [83]:
conn = get_db_connection(DB_NAME)
cursor = conn.cursor()

In [84]:
customer_df = pd.read_sql_query("""
    SELECT c.id, a.created_at, c.address
    FROM customer as c
    JOIN account as a
    ON c.account_id = a.id
    WHERE a.status != 'banned';
""", conn)

customer_df.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_25292\3640284414.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_df = pd.read_sql_query("""


,id,created_at,address
0,0,2023-12-19,"Phường Phong Hải, Thị xã Quảng Yên, Tỉnh Quảng..."
1,1,2023-09-08,"Xã Thạnh Bắc, Huyện Tân Biên, Tỉnh Tây Ninh"
2,2,2024-02-13,"Xã Thanh Xuân, Huyện Thanh Hà, Tỉnh Hải Dương"
3,3,2025-01-09,"Xã Tân Cương, Thành phố Thái Nguyên, Tỉnh Thái..."
4,4,2025-01-10,"Xã Tân Nghĩa, Huyện Cao Lãnh, Tỉnh Đồng Tháp"


Lấy ngẫu nhiên khách hàng để gán cho feedback

In [85]:
num_of_feedbacks = feedbacks.shape[0]

random.seed(RANDOM_SEED)

customer_sample = customer_df.sample(num_of_feedbacks, random_state=RANDOM_SEED)
customer_sample.reset_index(drop=True, inplace=True)

customer_ids_sample = customer_sample['id'].tolist()
account_created_at_sample = list(map(
    lambda x: x.strftime('%Y-%m-%d'), 
    customer_sample['created_at'].tolist()
    ))

Tạo từ điển `old_customer_id_to_new_customer_id`, ánh xạ từ customer_id cũ sang customer_id mới.

In [86]:
old_customer_id_to_new_customer_id = {}

Tạo từ điển `old_feedback_id_to_feedback` để ánh xạ từ `phiên bản` sang `id của biến thể` (phục vụ tạo bảng `feedback_manager` sau này)

In [87]:
old_feedback_id_to_feedback = {}

Tạo dataframe `feeback` (id, customer_id, product_id, product_variant_id, rating, comment, created_at)

In [88]:
feedback_rows = []

random.seed(RANDOM_SEED)

for index, row in tqdm(feedbacks.iterrows(), total=feedbacks.shape[0], desc='Processing feedbacks', unit="rows", colour='green'):
    id = index + 1

    customer_id = old_customer_id_to_new_customer_id.get(
        row['customer_id'], random.choice(customer_ids_sample)
    )
    
    old_customer_id_to_new_customer_id.setdefault(row['customer_id'], customer_id)

    product_id = mapping_old_product_id_to_new_product_id(row['product_id'])

    variant_id = mapping_option_to_variant_id(row['product_id'], row['variant'])

    rating = row['rating']

    comment = row['content']

    created_at = random_date(
        random.choice(account_created_at_sample), 
    )

    feedback_row = {
        'id': id,
        'customer_id': customer_id,
        'product_id': product_id,
        'product_variant_id': variant_id,
        'rating': rating,
        'comment': comment,
        'created_at': created_at,
    }

    feedback_rows.append(feedback_row)

    old_feedback_id_to_feedback.setdefault(int(row['feedback_id']), feedback_row)

feedback_df = pd.DataFrame(feedback_rows)

Processing feedbacks: 100%|██████████| 1934/1934 [00:00<00:00, 10624.85rows/s]


In [89]:
feedback_df.head()

,id,customer_id,product_id,product_variant_id,rating,comment,created_at
0,1,74600,2083,NaN,5,Đã mua gói hàng rất cẩn thận cảm thấy hài lòng...,2025-05-01 00:00:00
1,2,318965,2083,NaN,5,"Sản phẩm dễ cài đặt, nhiều tính năng, độ phân ...",2025-03-20 00:00:00
2,3,610704,2083,NaN,5,"hình rõ, app trên Android lẫn iOS chưa ổn định...",2025-02-05 00:00:00
3,4,347310,2083,NaN,5,NaN,2025-04-07 00:00:00
4,5,650534,2083,NaN,5,"giá rẻ, chất lượng tốtgiá rẻ, chất lượng tốtgi...",2024-03-31 00:00:00


### 2. Tạo dataframe `feeback_response` (id, manager_id, feedback_id, content, created_at)

#### 2.1. Tiền xử lý

In [90]:
feedback_response_file_name = {
    'camera_response': 'camera_fb_ma.csv',
    'dienthoai_response': 'dienthoai_fb_ma.csv',
    'mayban_response': 'dienthoaiban_fb_ma.csv',
    'cucgach_response': 'dienthoaiphothong_fb_ma.csv',
    'dieuhoa_response': 'dieuhoa_fb_ma.csv',
    'laptop_response': 'laptop_fb_ma.csv',
    'mayanh_response': 'mayanh_fb_ma.csv',
    'maydocsach_response': 'maydocsach_fb_ma.csv',
    'maygiat_response': 'maygiat_fb_ma.csv',
    'maytinhbang_response': 'maytinhbang_fb_ma.csv',
    'pc_response': 'maytinhdeban_fb_ma.csv',
    'tivi_response': 'tivi_fb_ma.csv',
    'tulanh_response': 'tulanh_fb_ma.csv',
}

feedback_response_dataframes = {
    name: pd.read_csv(f'./feedback_manager/{filename}', encoding='utf-8')
    for name, filename in feedback_response_file_name.items()
}

feedback_response_dataframes = {
    name: remove_duplicates(df)
    for name, df in feedback_response_dataframes.items()
}

Remove 2/89 duplicates
Remove 120/434 duplicates
Remove 0/1 duplicates
Remove 0/14 duplicates
Remove 0/0 duplicates
Remove 0/6 duplicates
Remove 0/1 duplicates
Remove 0/53 duplicates
Remove 0/5 duplicates
Remove 26/64 duplicates
Remove 0/0 duplicates
Remove 0/27 duplicates
Remove 0/3 duplicates


Gộp các feedback thành 1 dataframe duy nhất

In [91]:
feedback_responses = pd.concat([
    df for df in feedback_response_dataframes.values()
], ignore_index=True)

feedback_responses.head()

,id,feedback_id,manager_id,manager_name,content,time
0,3310469,20089032,1,Tiki Trading,Cảm ơn bạn đã tin tưởng Tiki và cho Tiki 5⭐️! ...,2025-02-28 21:09:02
1,3313040,20093906,1,Tiki Trading,Tiki rất vui khi nhận được đánh giá 5⭐️từ bạn....,2025-03-10 15:51:53
2,3308007,20083515,1,Tiki Trading,Cảm ơn bạn đã để lại nhận xét tại Tiki. Hy vọn...,2025-02-18 00:10:24
3,3306129,20079569,1,Tiki Trading,Cảm ơn bạn đã để lại nhận xét tại Tiki. Hy vọn...,2025-02-11 12:47:50
4,3306017,20078193,1,Tiki Trading,Cảm ơn bạn đã để lại nhận xét tại Tiki. Hy vọn...,2025-02-08 22:02:36


#### 2.2. Tạo dataframe

Tìm ra các nhân viên chăm sóc khách hàng

In [92]:
service_customer_df = pd.read_sql_query("""
    SELECT m.id as id
    FROM manager as m
    JOIN role as r
    ON m.role_id = r.id
    WHERE r.name = 'service_customer';
""", conn)

service_customer_df.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_25292\2899648652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  service_customer_df = pd.read_sql_query("""


,id
0,4
1,5
2,6
3,18
4,20


In [93]:
service_customer_ids = service_customer_df['id'].tolist()

Thay thế toàn bộ từ 'Tiki' xuất hiện trong nội dung phản hồi bằng 'PTIT-EShop'



In [94]:
feedback_responses['content'] = feedback_responses['content'].str.replace('tiki', 'PTIT-EShop', case=False)
feedback_responses['content'].head()

0    Cảm ơn bạn đã tin tưởng PTIT-EShop và cho PTIT...
1    PTIT-EShop rất vui khi nhận được đánh giá 5⭐️t...
2    Cảm ơn bạn đã để lại nhận xét tại PTIT-EShop. ...
3    Cảm ơn bạn đã để lại nhận xét tại PTIT-EShop. ...
4    Cảm ơn bạn đã để lại nhận xét tại PTIT-EShop. ...
Name: content, dtype: object

Tạo hàm `mapping_old_feedback_id_to_feedback` lấy phản hồi của khách hàng khi biết mã phản hồi của bảng cũ

In [95]:
def mapping_old_feedback_id_to_feedback(fb_id):
    return old_feedback_id_to_feedback.get(fb_id, None)

Tạo dataframe `feedback_response` (id, manager_id, feedback_id, content, created_at)

In [96]:
feedback_responses.columns

Index(['id', 'feedback_id', 'manager_id', 'manager_name', 'content', 'time'], dtype='object')

In [97]:
feedback_response_rows = []

random.seed(RANDOM_SEED)

for index, row in tqdm(feedback_responses.iterrows(), total=feedback_responses.shape[0], desc='Processing feedback responses', unit="rows", colour='green'):
    
    id = index + 1
    
    manager_id = random.choice(service_customer_ids)

    customer_feedback = mapping_old_feedback_id_to_feedback(row['feedback_id'])

    feedback_id = customer_feedback['id'] if customer_feedback else None

    comment = row['content']

    create_at = random_date(
        customer_feedback['created_at'] if customer_feedback else datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    )

    feedback_response_rows.append({
        'id': id,
        'manager_id': manager_id,
        'feedback_id': feedback_id,
        'comment': comment,
        'created_at': create_at,
    })

feedback_response_df = pd.DataFrame(feedback_response_rows)


Processing feedback responses: 100%|██████████| 549/549 [00:00<00:00, 8517.57rows/s]


In [98]:
feedback_response_df.head()

,id,manager_id,feedback_id,comment,created_at
0,1,132,4,Cảm ơn bạn đã tin tưởng PTIT-EShop và cho PTIT...,2025-04-10 00:00:00
1,2,5,31,PTIT-EShop rất vui khi nhận được đánh giá 5⭐️t...,2024-08-08 00:00:00
2,3,57,32,Cảm ơn bạn đã để lại nhận xét tại PTIT-EShop. ...,2024-04-09 00:00:00
3,4,30,33,Cảm ơn bạn đã để lại nhận xét tại PTIT-EShop. ...,2024-01-16 00:00:00
4,5,138,34,Cảm ơn bạn đã để lại nhận xét tại PTIT-EShop. ...,2025-03-29 00:00:00


# III. Xử lý giảm giá

Tạo hàm tạo giảm giá, với discount $\in [0.05, 0.3]$ và lợi nhuận tối thiểu bằng $0.05$

In [99]:
def get_discount_value(type: str, original_price: float, price: float, min_discount=0.05, max_discount=0.3, min_profit=0.05):
    valid_discounts = []
    
    for i in range(int(min_discount * 100), int(max_discount * 100) + 1):
        discount_rate = i / 100
        discounted_price = price * (1 - discount_rate)

        if discounted_price > original_price * (1 + min_profit):
            valid_discounts.append({
                'rate': discount_rate,
                'price': discounted_price,
            })
    
    if valid_discounts:
        if type == 'Percentage':
            return round(random.choice(valid_discounts)['rate'], 3)
        elif type == 'FixedAmount':
            return round(random.choice(valid_discounts)['price'], 3)
    
    return round(price, 3)

def check_status(start_date: str, end_date: str, current_date: str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')):
    start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

    current_date = datetime.strptime(current_date, '%Y-%m-%d %H:%M:%S')

    if start_date <= current_date <= end_date:
        return 'Active'
    elif current_date < start_date:
        return 'Inactive'
    else:
        return 'Expired'

In [100]:
types = ['Percentage', 'FixedAmount']

voucher_name = [
    'Giảm giá sinh nhật',
    'Giảm giá ngày lễ',
    'Giảm giá hot',
    'Giảm giá sốc',
    'Giảm giá cực mạnh',
    'Giảm giá lớn',
    'Giảm giá hấp dẫn',
    'Giảm giá cực chất',
    'Giảm giá không thể bỏ qua',
    'Giảm giá cực đã',
    'Giảm giá cực phê',
    'Giảm giá cực chất',
    'Giảm giá cực đỉnh',
    'Giảm giá cực chất lượng',
    'Giảm giá cực chất lượng cao'
]

Tạo từ điển `variant_id_to_voucher` để ánh xạ từ id phiên bản sang loại giảm giá

In [101]:
variant_id_to_voucher = {}

Cần xây dựng dataframe `discount`(`id`, `product_variant_id`, `code`, `name`, `type`, `value`, `status`, `start_date`, `end_date`)

In [102]:
discount_rows = []

random.seed(RANDOM_SEED)

random_product_varriant = product_variant_df[['id', 'price', 'original_price', 'profit']].sample(n=1000, replace=False, random_state=RANDOM_SEED)

for index, row in tqdm(random_product_varriant.iterrows(), total=random_product_varriant.shape[0], desc='Processing discounts', unit="rows", colour='green'):
    
    id = index + 1

    random_string = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=10))

    product_variant_id = row['id']

    code = f"PTIT-{random_string}"

    discount_name = random.choice(voucher_name)

    discount_type = random.choice(types)

    discount_value = get_discount_value(discount_type, row['original_price'], row['price'])

    start_date = random_date('2024-01-01', '2025-12-30')

    end_date = random_date(start_date, '2025-12-31')

    status = check_status(start_date, end_date)

    discount_rows.append({
        'id': id,
        'product_variant_id': product_variant_id,
        'code': code,
        'name': discount_name,
        'type': discount_type,
        'value': discount_value,
        'status': status,
        'start_date': start_date,
        'end_date': end_date,
    })

    variant_id_to_voucher[product_variant_id] = {
        'type': discount_type, 
        'value': discount_value, 
        'start_date': start_date, 
        'end_date': end_date
    }

discount_df = pd.DataFrame(discount_rows)

Processing discounts: 100%|██████████| 1000/1000 [00:00<00:00, 4397.00rows/s]


In [103]:
discount_df.head()

,id,product_variant_id,code,name,type,value,status,start_date,end_date
0,3586,3586,PTIT-XAJI0Y6DPB,Giảm giá sốc,Percentage,0.09,Inactive,2025-09-08 00:00:00,2025-09-11 00:00:00
1,1069,1069,PTIT-UZZPQK51FP,Giảm giá cực mạnh,Percentage,0.06,Expired,2024-12-10 00:00:00,2025-01-31 00:00:00
2,2045,2045,PTIT-DD4V30T9NT,Giảm giá cực chất lượng,FixedAmount,2912000.00,Expired,2024-07-15 00:00:00,2024-09-24 00:00:00
3,788,788,PTIT-BIKCIDKWNN,Giảm giá sốc,FixedAmount,17499000.00,Inactive,2025-09-15 00:00:00,2025-12-05 00:00:00
4,799,799,PTIT-G0FN9XUY41,Giảm giá sốc,Percentage,0.07,Active,2025-02-14 00:00:00,2025-07-01 00:00:00


# IV. Xử lý đơn hàng và các dataframe liên quan

- Dataframe `order` (`id`, `customer_id`, `order_date`, `shipping_address`, `status`, `payment_method`, `payment_date`, `payment_status`, `payment_amount`)

- Dataframe `order_item` (`id`, `product_variant_id`, `order_id`, `quantity`, `unit_price`, `note`)

- Dataframe `order_history` (`id`, `manager_id`, `order_id`, `processing_time`, `previous_status`, `new_status`)

## 1. Tiền xử lý

- Trạng thái đơn hàng `order_status` nhận 1 trong các giá trị: `Processing`, `Cancelled`, `Rejected`, `Completed`
- Trạng thái quản lý đơn hàng `manage_order_status_list` nhận 1 trong các giá trị: `Pending`, `Processing`, `Cancelled`, `Completed`
- Trạng thái thanh toán `payment_status` nhận 1 trong các giá trị: `Pending`, `Paid`, `Refunded`, `Partially Paid`
- Phương thức thanh toán `payment_method` nhận 1 trong các giá trị: `COD`, `Credit Card`, `Bank Transfer`, `Paypal`

Giữa phần tử của tập này và các phần tử của tập khác có mối quan hệ logic với nhau.

- `COD`: Khi phương thức thanh toán là COD, trạng thái thanh toán có thể là `Pending` khi đơn hàng ở trạng thái `Pending` hoặc `Processing`. Nếu đơn hàng bị hoàn trả, thanh toán có thể là Cancelled hoặc Refunded.

- `Credit Card`, `Bank Transfer`, và `PayPal`: Các phương thức này có thể có trạng thái thanh toán là `Partially Paid` hoặc `Refunded` tùy vào trạng thái của đơn hàng. Trong các trường hợp hoàn thành, trạng thái thanh toán sẽ là `Refunded` cho phương thức thẻ tín dụng và chuyển khoản ngân hàng.

- `Pending`: Khi đơn hàng ở trạng thái `Pending`, trạng thái thanh toán sẽ là `Pending` hoặc `Partially Paid`, tùy theo phương thức thanh toán.

- `Processing`: Nếu đơn hàng đang xử lý (`Processing`), trạng thái thanh toán có thể là `Partially Paid` nếu thanh toán chưa hoàn tất.

- `Completed`: Khi đơn hàng hoàn thành (`Completed`), trạng thái thanh toán sẽ là `Paid` nếu phương thức thanh toán là `COD`, hoặc `Refunded` cho các phương thức khác.

- `Cancelled`: Đơn hàng bị hủy sẽ có trạng thái thanh toán là `Cancelled` cho `COD`, hoặc `Refunded` cho các phương thức khác.

In [104]:
def get_order_status(manage_order_status, payment_status=None):
    if manage_order_status == 'Pending':
        return 'Processing'
    
    if manage_order_status == 'Rejected':
        return 'Rejected'
    
    if manage_order_status == 'Processing':
        if payment_status == 'Paid' or payment_status == 'Partially Paid':
            return 'Processing'
        
        return 'Rejected'

    if manage_order_status == 'Completed':
        if payment_status == 'Paid':
            return 'Completed'
        
        return 'Cancelled'
    
    return 'Processing'

def get_payment_status(payment_method, manage_order_status, order_status):
    if order_status == 'Rejected':
        return 'Cancelled' if payment_method == 'COD' else 'Refunded'
    
    if manage_order_status == 'Pending':
        return 'Pending'
    
    if manage_order_status == 'Processing':
        if payment_method == 'COD':
            return 'Pending'
        
        return 'Partially Paid'
    
    if manage_order_status == 'Completed':
        if payment_method == 'COD':
            return 'Paid'
        
        return 'Refunded'
    
    if manage_order_status == 'Cancelled':
        return 'Cancelled' if payment_method == 'COD' else 'Refunded'
    
    return 'Pending'

def random_order(payment_method=None, manage_order_status=None, order_status=None, payment_status=None):
    if payment_method is None:
        payment_method = random.choice(['COD', 'Credit Card', 'Bank Transfer', 'PayPal'])
    
    if manage_order_status is None:
        manage_order_status = random.choice(['Pending', 'Processing', 'Cancelled', 'Completed'])

    if order_status is None:
        order_status = get_order_status(manage_order_status, payment_status)

    if payment_status is None:
        payment_status = get_payment_status(payment_method, manage_order_status, order_status)

    return {
        'payment_method': payment_method,
        'manage_order_status': manage_order_status,
        'order_status': order_status,
        'payment_status': payment_status
    }

Chọn ra các khách hàng từ dataframe `customer` để lập đơn hàng. Tuy nhiên, đối với các khách hàng đã từng gửi phản hồi về sản phẩm, thì bắt buộc phải chọn để tạo đơn hàng và lưu trữ hóa đơn từ họ

In [105]:
feedback_customers = feedback_df[['customer_id', 'created_at', 'product_id', 'product_variant_id']].rename(
    columns={
        'created_at': 'feedback_created_at',
    }
)

feedback_customers.head()

,customer_id,feedback_created_at,product_id,product_variant_id
0,74600,2025-05-01 00:00:00,2083,NaN
1,318965,2025-03-20 00:00:00,2083,NaN
2,610704,2025-02-05 00:00:00,2083,NaN
3,347310,2025-04-07 00:00:00,2083,NaN
4,650534,2024-03-31 00:00:00,2083,NaN


In [106]:
feedback_customers = feedback_customers.merge(
    customer_df[['id', 'address']], how = 'left', left_on='customer_id', right_on='id'
).drop(columns=['id'])

feedback_customers.head()

,customer_id,feedback_created_at,product_id,product_variant_id,address
0,74600,2025-05-01 00:00:00,2083,NaN,"Phường Hòa Xuân, Quận Cẩm Lệ, Thành phố Đà Nẵng"
1,318965,2025-03-20 00:00:00,2083,NaN,"Thị trấn Vị Xuyên, Huyện Vị Xuyên, Tỉnh Hà Giang"
2,610704,2025-02-05 00:00:00,2083,NaN,"Xã Nhơn Ninh, Huyện Tân Thạnh, Tỉnh Long An"
3,347310,2025-04-07 00:00:00,2083,NaN,"Phường Phổ Minh, Thị xã Đức Phổ, Tỉnh Quảng Ngãi"
4,650534,2024-03-31 00:00:00,2083,NaN,"Xã Long Khánh, Huyện Bến Cầu, Tỉnh Tây Ninh"


In [107]:
customer_sample = customer_df[['id', 'created_at', 'address']] \
            .sample(n=300000, random_state=RANDOM_SEED) \
            .rename(columns={
                'id': 'customer_id',
                'created_at': 'customer_created_at',
            })

customer_sample.head()

,customer_id,customer_created_at,address
128449,142720,2023-01-17,"Xã Minh Đức, Huyện Mỏ Cày Nam, Tỉnh Bến Tre"
750552,834084,2024-12-12,"Xã Dân Quyền, Huyện Triệu Sơn, Tỉnh Thanh Hóa"
254582,282885,2023-12-07,"Xã Văn Minh, Huyện Na Rì, Tỉnh Bắc Kạn"
679837,755508,2024-07-04,"Phường Vân Dương, Thành phố Bắc Ninh, Tỉnh Bắc..."
835088,927949,2023-08-08,"Xã Hòa Minh, Huyện Tuy Phong, Tỉnh Bình Thuận"


In [108]:
customer_order_df = pd.concat([
    feedback_customers, customer_sample
], axis=0, ignore_index=True)

customer_order_df.head()

,customer_id,feedback_created_at,product_id,product_variant_id,address,customer_created_at
0,74600,2025-05-01 00:00:00,2083.0,NaN,"Phường Hòa Xuân, Quận Cẩm Lệ, Thành phố Đà Nẵng",NaT
1,318965,2025-03-20 00:00:00,2083.0,NaN,"Thị trấn Vị Xuyên, Huyện Vị Xuyên, Tỉnh Hà Giang",NaT
2,610704,2025-02-05 00:00:00,2083.0,NaN,"Xã Nhơn Ninh, Huyện Tân Thạnh, Tỉnh Long An",NaT
3,347310,2025-04-07 00:00:00,2083.0,NaN,"Phường Phổ Minh, Thị xã Đức Phổ, Tỉnh Quảng Ngãi",NaT
4,650534,2024-03-31 00:00:00,2083.0,NaN,"Xã Long Khánh, Huyện Bến Cầu, Tỉnh Tây Ninh",NaT


Chọn ra các biến thể sản phẩm từ dataframe `product_variant` để lập đơn hàng. Tuy nhiên, đối với các khách hàng đã từng gửi phản hồi về sản phẩm, thì bắt buộc phải chọn sản phẩm mà họ từng mua để tạo đơn hàng và lưu trữ hóa đơn từ họ

Lấy mẫu ngẫu nhiên các id biến thể

In [109]:
random.seed(RANDOM_SEED)

product_variant_ids_sample = random.choices(
    product_variant_df['id'].tolist(), k=300000
)

Tạo ánh xạ từ ``product_id`` sang ``product_variant_id``

In [110]:
product_id_to_variant_id = product_variant_df.groupby('product_id')['id'].apply(list).to_dict()

Hoàn thiện sinh ngẫu nhiên/ gán id biến thể cho `customer`

In [111]:
def fill_product_variant_id(row):

    if pd.isna(row['product_variant_id']):
        if pd.isna(row['product_id']):
            return int(random.choice(product_variant_ids_sample))
        
        return int(random.choice(product_id_to_variant_id[row['product_id']]))
    
    return int(row['product_variant_id'])

random.seed(RANDOM_SEED)

customer_order_df['product_variant_id'] = customer_order_df.apply(
    lambda row: fill_product_variant_id(row), axis=1
)

customer_order_df.drop(columns=['product_id'], inplace=True)

customer_order_df['customer_created_at'] = customer_order_df['customer_created_at'].astype(str).str.strip()

In [112]:
customer_order_df.head()

,customer_id,feedback_created_at,product_variant_id,address,customer_created_at
0,74600,2025-05-01 00:00:00,2459,"Phường Hòa Xuân, Quận Cẩm Lệ, Thành phố Đà Nẵng",NaT
1,318965,2025-03-20 00:00:00,2459,"Thị trấn Vị Xuyên, Huyện Vị Xuyên, Tỉnh Hà Giang",NaT
2,610704,2025-02-05 00:00:00,2459,"Xã Nhơn Ninh, Huyện Tân Thạnh, Tỉnh Long An",NaT
3,347310,2025-04-07 00:00:00,2459,"Phường Phổ Minh, Thị xã Đức Phổ, Tỉnh Quảng Ngãi",NaT
4,650534,2024-03-31 00:00:00,2459,"Xã Long Khánh, Huyện Bến Cầu, Tỉnh Tây Ninh",NaT


Lấy `id` của các nhân viên quản lý bán hàng (Để phục vụ tạo bảng `order_history`)

In [113]:
manager_df = pd.read_sql_query("""
    SELECT m.id
    FROM manager as m
    JOIN role as r
    ON m.role_id = r.id
    WHERE r.name = 'product_manager';
""", conn)

manager_df.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_25292\3802231901.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  manager_df = pd.read_sql_query("""


,id
0,0
1,8
2,11
3,14
4,15


In [114]:
manager_ids = manager_df['id'].tolist()

Xử lý ngày tạo đơn

- Nếu khách hàng đã đánh giá sản phẩm, ngày tạo đơn có thể nằm từ 1-3 ngày trước khi đánh giá.

- Nếu khách hàng chưa đánh giá, sinh ngẫu nhiên ngày trong khoảng từ khi tạo tài khoản tới hiện tại.

In [115]:
def get_order_date(row):
    if pd.isna(row['feedback_created_at']):
        return random_date(row['customer_created_at'])
    
    return (datetime.strptime(row['feedback_created_at'], '%Y-%m-%d %H:%M:%S') - timedelta(days=random.randint(1, 3))).strftime('%Y-%m-%d %H:%M:%S')

In [116]:
def get_status(row):
    if pd.isna(row['feedback_created_at']):
        return random_order()
    
    return random_order(manage_order_status='Completed', order_status='Completed', payment_status='Paid')

In [117]:
def get_payment_date(row, payment_method, order_date, order_status):
    if pd.isna(row['feedback_created_at']):
        if payment_method == 'COD' and order_status == 'Completed':
                return random_date(order_date)
        if payment_method != 'COD':
            return order_date
        
        return None
    
    if payment_method == 'COD':
        return random_date(order_date)
    
    return order_date

In [118]:
def get_payment_amount(unit_price, quantity, order_date, voucher):

    if voucher is None or check_status(voucher['start_date'], voucher['end_date'], order_date) == 'Expired':
        return unit_price * quantity
    

    if voucher['type'] == 'Percentage':
        return unit_price * quantity * (1 - voucher['value'])
    
    return unit_price * quantity - voucher['value']

## 2. Tạo dataframe ``order``, ``order_item``, ``order_history``

- Dataframe `order` (`id`, `customer_id`, `order_date`, `shipping_address`, `status`, `payment_method`, `payment_date`, `payment_status`, `payment_amount`)

- Dataframe `order_item` (`id`, `product_variant_id`, `order_id`, `quantity`, `unit_price`, `note`)

- Dataframe `order_history` (`id`, `manager_id`, `order_id`, `processing_time`, `previous_status`, `new_status`)

In [119]:
order_rows = []
order_item_rows = []
order_history_rows = []

random.seed(RANDOM_SEED)

for index, row in tqdm(customer_order_df.iterrows(), total=customer_order_df.shape[0], desc='Processing orders', unit="rows", colour='green'):
    id = index + 1

    customer_id = row['customer_id']

    order_date = get_order_date(row)

    shipping_address = row['address']

    status_dict = get_status(row)

    status = status_dict['order_status']

    payment_method = status_dict['payment_method']

    payment_date = get_payment_date(row, payment_method, order_date, status)

    payment_status = status_dict['payment_status']

    product_variant_id = row['product_variant_id']

    quantity = random.randint(1, 5)

    unit_price = product_variant_df.loc[product_variant_df['id'] == product_variant_id, 'price'].values[0]

    voucher = variant_id_to_voucher.get(product_variant_id, None)

    payment_amount = get_payment_amount(unit_price, quantity, order_date, voucher)

    manager_id = random.choice(manager_ids)

    processing_time = random_date(order_date, payment_date) if payment_date else None

    previous_status = random.choice(['Pending', 'Processing', 'Completed', 'Cancelled'])

    new_status = status

    order_rows.append({
        'id': id,
        'customer_id': customer_id,
        'order_date': order_date,
        'shipping_address': shipping_address,
        'status': status,
        'payment_method': payment_method,
        'payment_date': payment_date,
        'payment_status': payment_status,
        'payment_amount': payment_amount,
    })

    order_item_rows.append({
        'id': id,
        'product_variant_id': product_variant_id,
        'order_id': id,
        'quantity': quantity,
        'unit_price': unit_price,
        'note': '',
    })

    order_history_rows.append({
        'id': id,
        'manager_id': manager_id,
        'order_id': id,
        'processing_time': processing_time,
        'previous_status': previous_status,
        'new_status': new_status,
    })

order_df = pd.DataFrame(order_rows)
order_item_df = pd.DataFrame(order_item_rows)
order_history_df = pd.DataFrame(order_history_rows)

Processing orders: 100%|██████████| 301934/301934 [01:43<00:00, 2924.14rows/s]


In [120]:
order_df.head()

,id,customer_id,order_date,shipping_address,status,payment_method,payment_date,payment_status,payment_amount
0,1,74600,2025-04-28 00:00:00,"Phường Hòa Xuân, Quận Cẩm Lệ, Thành phố Đà Nẵng",Completed,COD,2025-04-28 00:00:00,Paid,959310.0
1,2,318965,2025-03-17 00:00:00,"Thị trấn Vị Xuyên, Huyện Vị Xuyên, Tỉnh Hà Giang",Completed,COD,2025-04-29 00:00:00,Paid,1877310.0
2,3,610704,2025-02-04 00:00:00,"Xã Nhơn Ninh, Huyện Tân Thạnh, Tỉnh Long An",Completed,COD,2025-02-15 00:00:00,Paid,500310.0
3,4,347310,2025-04-04 00:00:00,"Phường Phổ Minh, Thị xã Đức Phổ, Tỉnh Quảng Ngãi",Completed,Credit Card,2025-04-04 00:00:00,Paid,1877310.0
4,5,650534,2024-03-28 00:00:00,"Xã Long Khánh, Huyện Bến Cầu, Tỉnh Tây Ninh",Completed,Bank Transfer,2024-03-28 00:00:00,Paid,41310.0


In [121]:
order_item_df.head()

,id,product_variant_id,order_id,quantity,unit_price,note
0,1,2459,1,3,459000,
1,2,2459,2,5,459000,
2,3,2459,3,2,459000,
3,4,2459,4,5,459000,
4,5,2459,5,1,459000,


In [122]:
order_history_df.head()

,id,manager_id,order_id,processing_time,previous_status,new_status
0,1,95,1,2025-04-28 00:00:00,Processing,Completed
1,2,35,2,2025-04-23 00:00:00,Cancelled,Completed
2,3,91,3,2025-02-12 00:00:00,Pending,Completed
3,4,171,4,2025-04-04 00:00:00,Cancelled,Completed
4,5,61,5,2024-03-28 00:00:00,Completed,Completed


# V. Lưu Dataframe vào SQL Server

Xóa các bảng nếu đã tồn tại trong database

In [123]:
conn.execute("""
    IF OBJECT_ID('order_history', 'U') IS NOT NULL DROP TABLE order_history;

    IF OBJECT_ID('order_item', 'U') IS NOT NULL DROP TABLE order_item;

    IF OBJECT_ID('order', 'U') IS NOT NULL DROP TABLE [order];   

    IF OBJECT_ID('discount', 'U') IS NOT NULL DROP TABLE discount;

    IF OBJECT_ID('feedback_response', 'U') IS NOT NULL DROP TABLE feedback_response;

    IF OBJECT_ID('feedback', 'U') IS NOT NULL DROP TABLE feedback;

    IF OBJECT_ID('attribute_variant', 'U') IS NOT NULL DROP TABLE attribute_variant;

    IF OBJECT_ID('product_variant', 'U') IS NOT NULL DROP TABLE product_variant;

    IF OBJECT_ID('attribute_value', 'U') IS NOT NULL DROP TABLE attribute_value;

    IF OBJECT_ID('attribute', 'U') IS NOT NULL DROP TABLE attribute;

    IF OBJECT_ID('product', 'U') IS NOT NULL DROP TABLE product;

    IF OBJECT_ID('category', 'U') IS NOT NULL DROP TABLE category; 
""")

Tạo bảng

In [124]:
conn.execute("""
    CREATE TABLE category (
        id INT PRIMARY KEY IDENTITY(1,1),
        name NVARCHAR(255) NOT NULL,
    )
""")

conn.execute("""
    CREATE TABLE product (
        id INT PRIMARY KEY IDENTITY(1,1),
        category_id INT NOT NULL,
        name NVARCHAR(255) NOT NULL,
        description NVARCHAR(MAX),
        specification NVARCHAR(MAX),
        image_url NVARCHAR(MAX),
        brand NVARCHAR(255),
        CONSTRAINT fk_product_category FOREIGN KEY (category_id) REFERENCES category(id)
    )
""")

conn.execute("""
    CREATE TABLE attribute (
        id INT PRIMARY KEY IDENTITY(1,1),
        name NVARCHAR(255) NOT NULL
    )
""")

conn.execute("""
    CREATE TABLE attribute_value (
        id INT PRIMARY KEY IDENTITY(1,1),
        attribute_id INT NOT NULL,
        value NVARCHAR(255) NOT NULL,
        CONSTRAINT fk_attribute_value_attribute FOREIGN KEY (attribute_id) REFERENCES attribute(id)
    )
""")

conn.execute("""
    CREATE TABLE product_variant (
        id INT PRIMARY KEY IDENTITY(1,1),
        product_id INT NOT NULL,
        price DECIMAL(30,3) NOT NULL,
        original_price DECIMAL(30,3) NOT NULL,
        profit DECIMAL(30,3) NOT NULL,
        sku NVARCHAR(MAX),
        stock_quantity INT DEFAULT 0,
        sold_quantity INT DEFAULT 0,
        CONSTRAINT fk_product_variant_product FOREIGN KEY (product_id) REFERENCES product(id)
    )
""")

conn.execute("""
    CREATE TABLE attribute_variant (
        product_variant_id INT NOT NULL,
        attribute_id INT NOT NULL,
        attribute_value_id INT NOT NULL,
        CONSTRAINT pk_variant_attribute PRIMARY KEY (attribute_value_id, attribute_id, product_variant_id),
        CONSTRAINT fk_variant_attribute_attribute_value FOREIGN KEY (attribute_value_id) REFERENCES attribute_value(id),
        CONSTRAINT fk_variant_attribute_attribute FOREIGN KEY (attribute_id) REFERENCES attribute(id),
        CONSTRAINT fk_variant_attribute_product_variant FOREIGN KEY (product_variant_id) REFERENCES product_variant(id)
    )
""")

conn.execute("""
    CREATE TABLE feedback (
        id INT PRIMARY KEY IDENTITY(1,1),
        customer_id INT NOT NULL,
        product_id INT NOT NULL,
        product_variant_id INT NULL,
        rating INT CHECK (rating BETWEEN 1 AND 5),
        comment NVARCHAR(MAX) NULL,
        created_at DATETIME DEFAULT GETDATE(),
        CONSTRAINT fk_feedback_product FOREIGN KEY (product_id) REFERENCES product(id),
        CONSTRAINT fk_feedback_product_variant FOREIGN KEY (product_variant_id) REFERENCES product_variant(id)
    )
""")

conn.execute("""
    CREATE TABLE feedback_response (
        id INT PRIMARY KEY IDENTITY(1,1),
        manager_id INT NOT NULL,
        feedback_id INT NOT NULL,
        content NVARCHAR(MAX),
        created_at DATETIME DEFAULT GETDATE(),
        CONSTRAINT fk_feedback_response_manager FOREIGN KEY (manager_id) REFERENCES manager(id),
        CONSTRAINT fk_feedback_response_feedback FOREIGN KEY (feedback_id) REFERENCES feedback(id)
    )
""")

conn.execute("""
    CREATE TABLE discount (
        id INT PRIMARY KEY IDENTITY(1,1),
        product_variant_id INT NOT NULL,
        code NVARCHAR(100) UNIQUE,
        name NVARCHAR(255),
        type NVARCHAR(50), 
        value DECIMAL(19,3),
        status NVARCHAR(50) DEFAULT 'Active',
        start_date DATETIME,
        end_date DATETIME,
        CONSTRAINT fk_discount_product_variant FOREIGN KEY (product_variant_id) REFERENCES product_variant(id)
    )
""")

conn.execute("""
    CREATE TABLE [order] (
        id INT PRIMARY KEY IDENTITY(1,1),
        customer_id INT NOT NULL,
        order_date DATETIME DEFAULT GETDATE(),
        shipping_address NVARCHAR(MAX),
        status NVARCHAR(50) DEFAULT 'Processing', 
        payment_method NVARCHAR(255),
        payment_date DATETIME,
        payment_status NVARCHAR(50) DEFAULT 'Pending',
        payment_amount DECIMAL(30,3),
        CONSTRAINT fk_order_customer FOREIGN KEY (customer_id) REFERENCES customer(id)
    )
""")

conn.execute("""
    CREATE TABLE order_item (
        id INT PRIMARY KEY IDENTITY(1,1),
        product_variant_id INT NOT NULL,
        order_id INT NOT NULL,
        quantity INT NOT NULL DEFAULT 1,
        unit_price DECIMAL(30,2) NOT NULL,
        note NVARCHAR(MAX),
        CONSTRAINT fk_order_item_product_variant FOREIGN KEY (product_variant_id) REFERENCES product_variant(id),
        CONSTRAINT fk_order_item_order FOREIGN KEY (order_id) REFERENCES [order](id)
    )
""")

conn.execute("""
    CREATE TABLE order_history (
        id INT PRIMARY KEY IDENTITY(1,1),
        manager_id INT NOT NULL,
        order_id INT NOT NULL,
        processing_time DATETIME DEFAULT GETDATE(),
        previous_status NVARCHAR(50), 
        new_status NVARCHAR(50), 
        CONSTRAINT fk_order_history_manager FOREIGN KEY (manager_id) REFERENCES manager(id),
        CONSTRAINT fk_order_history_order FOREIGN KEY (order_id) REFERENCES [order](id)
    )
""")

In [125]:
category_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(category_df.itertuples(index=False, name=None), total=category_df.shape[0], desc="Creating category tuples", unit="rows", colour="green")
]

product_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(product_df.itertuples(index=False, name=None), total=product_df.shape[0], desc="Creating product tuples", unit="rows", colour="green")
]

attribute_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(attribute_df.itertuples(index=False, name=None), total=attribute_df.shape[0], desc="Creating attribute tuples", unit="rows", colour="green")
]

attribute_value_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(attribute_value_df.itertuples(index=False, name=None), total=attribute_value_df.shape[0], desc="Creating attribute_value tuples", unit="rows", colour="green")
]

product_variant_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(product_variant_df.itertuples(index=False, name=None), total=product_variant_df.shape[0], desc="Creating product_variant tuples", unit="rows", colour="green")
]

attribute_variant_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(attribute_variant_df.itertuples(index=False, name=None), total=attribute_variant_df.shape[0], desc="Creating attribute_variant tuples", unit="rows", colour="green")
]

feedback_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(feedback_df.itertuples(index=False, name=None), total=feedback_df.shape[0], desc="Creating feedback tuples", unit="rows", colour="green")
]

feedback_response_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(feedback_response_df.itertuples(index=False, name=None), total=feedback_response_df.shape[0], desc="Creating feedback_response tuples", unit="rows", colour="green")
]

order_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(order_df.itertuples(index=False, name=None), total=order_df.shape[0], desc="Creating order tuples", unit="rows", colour="green")
]

order_item_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(order_item_df.itertuples(index=False, name=None), total=order_item_df.shape[0], desc="Creating order_item tuples", unit="rows", colour="green")
]

order_history_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(order_history_df.itertuples(index=False, name=None), total=order_history_df.shape[0], desc="Creating order_history tuples", unit="rows", colour="green")
]

discount_tuples = [
    tuple(None if pd.isna(x) else x for x in row)
    for row in tqdm(discount_df.itertuples(index=False, name=None), total=discount_df.shape[0], desc="Creating discount tuples", unit="rows", colour="green")
]

Creating discount tuples: 100%|██████████| 1000/1000 [00:00<00:00, 267204.18rows/s]


In [126]:
cursor.fast_executemany = True

In [127]:
cursor.execute("SET IDENTITY_INSERT category ON;")
cursor.executemany("INSERT INTO category (id, name) VALUES (?, ?)", category_tuples)
cursor.execute("SET IDENTITY_INSERT category OFF;")

In [128]:
cursor.execute("SET IDENTITY_INSERT product ON;")
cursor.executemany("INSERT INTO product (id, category_id, name, description, specification, image_url, brand) VALUES (?, ?, ?, ?, ?, ?, ?)", product_tuples)
cursor.execute("SET IDENTITY_INSERT product OFF;")

In [129]:
cursor.execute("SET IDENTITY_INSERT attribute ON;")
cursor.executemany("INSERT INTO attribute (id, name) VALUES (?, ?)", attribute_tuples)
cursor.execute("SET IDENTITY_INSERT attribute OFF;")

In [130]:
cursor.execute("SET IDENTITY_INSERT attribute_value ON;")
cursor.executemany("INSERT INTO attribute_value (id, attribute_id, value) VALUES (?, ?, ?)", attribute_value_tuples)
cursor.execute("SET IDENTITY_INSERT attribute_value OFF;")

In [131]:
cursor.execute("SET IDENTITY_INSERT product_variant ON;")
cursor.executemany("INSERT INTO product_variant (id, product_id, price, original_price, profit, sku, stock_quantity, sold_quantity) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", product_variant_tuples)
cursor.execute("SET IDENTITY_INSERT product_variant OFF;")

In [132]:
cursor.executemany("INSERT INTO attribute_variant (product_variant_id, attribute_id, attribute_value_id) VALUES (?, ?, ?)", attribute_variant_tuples)

In [133]:
cursor.execute("SET IDENTITY_INSERT feedback ON;")
cursor.executemany("INSERT INTO feedback (id, customer_id, product_id, product_variant_id, rating, comment, created_at) VALUES (?, ?, ?, ?, ?, ?, ?)", feedback_tuples)
cursor.execute("SET IDENTITY_INSERT feedback OFF;")

In [134]:
cursor.execute("SET IDENTITY_INSERT feedback_response ON;")
cursor.executemany("INSERT INTO feedback_response (id, manager_id, feedback_id, content, created_at) VALUES (?, ?, ?, ?, ?)", feedback_response_tuples)
cursor.execute("SET IDENTITY_INSERT feedback_response OFF;")

In [135]:
cursor.execute("SET IDENTITY_INSERT discount ON;")
cursor.executemany("INSERT INTO discount (id, product_variant_id, code, name, type, value, status, start_date, end_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", discount_tuples)
cursor.execute("SET IDENTITY_INSERT discount OFF;")

In [136]:
cursor.execute("SET IDENTITY_INSERT [order] ON;")
cursor.executemany("INSERT INTO [order] (id, customer_id, order_date, shipping_address, status, payment_method, payment_date, payment_status, payment_amount) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", order_tuples)
cursor.execute("SET IDENTITY_INSERT [order] OFF;")

In [137]:
cursor.execute("SET IDENTITY_INSERT order_item ON;")
cursor.executemany("INSERT INTO order_item (id, product_variant_id, order_id, quantity, unit_price, note) VALUES (?, ?, ?, ?, ?, ?)", order_item_tuples)
cursor.execute("SET IDENTITY_INSERT order_item OFF;")

In [138]:
cursor.execute("SET IDENTITY_INSERT order_history ON;")
cursor.executemany("INSERT INTO order_history (id, manager_id, order_id, processing_time, previous_status, new_status) VALUES (?, ?, ?, ?, ?, ?)", order_history_tuples)
cursor.execute("SET IDENTITY_INSERT order_history OFF;")

# VI. Đóng kết nối

In [140]:
conn.commit()
conn.close()

In [141]:
category_df.to_csv('./transformedData/category.csv', index=False, encoding='utf-8-sig')
attribute_value_df.to_csv('./transformedData/attribute_value.csv', index=False, encoding='utf-8-sig')
attribute_variant_df.to_csv('./transformedData/attribute_variant.csv', index=False, encoding='utf-8-sig')
attribute_df.to_csv('./transformedData/attribute.csv', index=False, encoding='utf-8-sig')
discount_df.to_csv('./transformedData/discount.csv', index=False, encoding='utf-8-sig')
feedback_response_df.to_csv('./transformedData/feedback_response.csv', index=False, encoding='utf-8-sig')
feedback_df.to_csv('./transformedData/feedback.csv', index=False, encoding='utf-8-sig')
order_history_df.to_csv('./transformedData/order_history.csv', index=False, encoding='utf-8-sig')
order_item_df.to_csv('./transformedData/order_item.csv', index=False, encoding='utf-8-sig')
order_df.to_csv('./transformedData/order.csv', index=False, encoding='utf-8-sig')
product_df.to_csv('./transformedData/product.csv', index=False, encoding='utf-8-sig')
product_variant_df.to_csv('./transformedData/product_variant.csv', index=False, encoding='utf-8-sig')